---
title: "基于Quarto和GitHub Pages的个人博客网站搭建流程"
author: "逃之夭夭"
date: "2026-01-25"
categories: [tools]
image: "image.jpg"
---

> 基于Github Actions自动渲染Quarto博客

本篇博客旨在提供完整的基于Quarto和GitHub Pages的个人博客网站搭建流程，帮助读者快速搭建个人博客系统。

>💡 在阅读之前，你应该已经：
>
>   安装quarto cli并了解其基本用法，[Get Started with quarto](https://quarto.org/docs/get-started/)

>💡 
>了解最详细的quarto官方关于搭建博客网站的教程，自行阅读并知晓每一种方式，或是查阅如何修改以定制一个更个性化的博客网站。
>
>请参考：[quarto website-blog tutorial](https://quarto.org/docs/websites/website-blog.html)
>
>一个有效建议是：
>   你总是应该在完成搭建博客网站后查阅官方文档，以知晓更加完整的信息。

| 工具 |
| :--- |
| cursor（或任何其他编程IDE，如vscode） |

## 创建Quarto Blog项目

1. 打开cursor命令面板，输入**Quarto:Create Project**创建项目

    ![](create_project.png)

2. 选择**Blog Project**

    ![](blog_project.png)

3. 选择一个文件夹作为博客项目的根目录

    ![](choose_path.png)
4. 为博客项目起名

    ![](create_blog_name.png)
5. quarto将会创建默认配置以支持博客网站功能

    ![](blog_init_state.png)
6. 点击右上角“预览”按钮，quarto将为你渲染博客网站

    ![](preview.png)

这是在入门博客项目中创建的关键文件摘要：

| 文件 | 描述 |
| :--- | :--- |
| `_quarto.yml` | Quarto 项目文件 |
| `index.qmd` | 博客主页 |
| `about.qmd` | 关于页面 |
| `posts/` | 包含文章的目录 |
| `posts/_metadata.yml` | `posts` 的共享选项 |
| `styles.css` | 网站的定制 CSS |

主页预览界面

![](preview_render.png)

## 项目文件介绍

### 主页

主页是关于posts目录下所有博客的文章列表，如主页预览界面所示。

源代码：
```YAML
---
title: "blog_tutorial"
listing:
  contents: posts
  feed: true
  sort: "date desc"
  type: default
  categories: true
  sort-ui: false
  filter-ui: false
page-layout: full
title-block-banner: true
---
```

| 字段 | 描述 |
| :--- | :--- |
| `title` | 标题 |
| `categories` | 开启博客分类 |

#### Categories（分类）

`categories: true`表示配置为启用分类，这些分类显示在主页的右侧边缘。

分类从包含在列表中的文档的前置内容中读取。例如，这里是一个包含分类的样本帖子元数据：
```YAML
---
title: "Post With Code"
description: "Post description"
author: "Fizz McPhee"
date: "5/22/2021"
categories:
  - news
  - code
  - analysis
---
```

#### 关于页面

`about.qmd`包含有关博客及其作者的更多信息。

`about.qmd`源代码可能的样子：
```markdown
---
title: "About"
image: profile.jpg
about:
  template: jolla
  links:
    - icon: twitter
      text: Twitter
      href: https://twitter.com
    - icon: linkedin
      text: LinkedIn
      href: https://linkedin.com
    - icon: github
      text: Github
      href: https://github.com
---

About this blog
```

#### posts文件夹

构成博客内容的帖子位于`posts`目录中。

通过在`posts`内创建一个子目录，并向该目录添加一个index.ipynb（或是 `index.qmd`）文件来为你的博客添加一篇新帖子。这个文件就是新的博客帖子，当你渲染(quarto render)它时，博客主页将自动更新，在列表顶部包含最新的帖子。

## 发布（Publishing）

有多种方式发布Quarto博客。本文仅介绍基于GitHub Action的GitHub Pages发布方式。

1. 在github创建新仓库

![](create_repo.png)

2. 本地项目初始化并关联github仓库
   1. 创建README.md文件，内容为`# 你的仓库名`
   2. 创建.gitignore文件
    ```markdown
    /.quarto/
    /_site/
    ```
   3. 修改`_quarto.yml`中`site-url`为你的仓库地址
   4. 提交修改并关联远端仓库
    ```bash
    git init
    git add .
    git commit -m "first commit"
    git branch -M main
    git remote add origin git@github.com:TheSecondStep/quarto-blog-tutorial.git
    git push -u origin main
    ```

3. 本地执行一次命令行`quarto publish gh-pages`
   `quarto`将为你创建`gh-pages`分支，用于GitHub Pages部署网站使用。
4. 部署GitHub Action
   GitHub Action可以支持在`main`分支修改上传代码到github仓库后，自动触发工作流，渲染博客网站并更新。

   创建`.github/workflows/publish.yml`，内容如下
   
    ```yml
    on:
      workflow_dispatch:
      push:
        branches: main

    name: Quarto Publish

    jobs:
      build-deploy:
        runs-on: ubuntu-latest
        permissions:
          contents: write
        steps:
          - name: Check out repository
            uses: actions/checkout@v4

          - name: Set up Quarto
            uses: quarto-dev/quarto-actions/setup@v2

          - name: Render and Publish
            uses: quarto-dev/quarto-actions/publish@v2
            with:
              target: gh-pages
            env:
              GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
   ```
5. 在github仓库设置的Actions部分的`Workflow permissions`勾选`Read and write permissions`。
6. 推送上述修改到github，后续只要手动修改博客并推送到main分支，将自动触发GitHub Action渲染网站。
   
   ![](action.png)

### 执行代码

GitHub Action也支持配置自动执行代码，来作为作为网站渲染的一部分。

>💡 确保你的代码能够在GitHub Action中执行成功的最佳实践是通过虚拟环境管理，如venv
>
>本文以uv管理venv虚拟环境为例，确保你的本地环境已经通过虚拟环境管理python代码且同步到GitHub

修改publish.yml，然后推送到远端分支

```yml
on:
  workflow_dispatch:
  push:
    branches: main

name: Quarto Publish

jobs:
  build-deploy:
    runs-on: ubuntu-latest
    permissions:
      contents: write
    steps:
      - name: Check out repository
        uses: actions/checkout@v4

      - name: Set up Quarto
        uses: quarto-dev/quarto-actions/setup@v2

      - name: Install uv
        uses: astral-sh/setup-uv@v5
        with:
          enable-cache: true
          python-version: '3.13' #你的虚拟环境的python版本

      - name: Install Dependencies
        run: |
          uv sync

      - name: Render and Publish
        uses: quarto-dev/quarto-actions/publish@v2
        with:
          target: gh-pages
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
          QUARTO_PYTHON: .venv/bin/python
```

## 添加评论区

>若思想没有回响，文字便只是一场永无止境的独白；评论区并非点缀，而是博客得以呼吸的肺叶。


1. 登录https://giscus.app/zh-CN 并**根据指引操作**， `giscus`是利用`GitHub Discussions`实现的评论系统，天然与我们这套基于Github的博客系统适配。
2. 然后我们会看到类似这么一段配置

    ![](giscus.png)
3. 修改`_quarto.yml`，加上对应配置

    ![](comments_config.png)

4. 上传所有修改，我们的网站就支持评论功能了

    ![](comments_view.png)

## 结语

至此，基于Quarto的个人网站已经部署上线，支持通过GitHub Pages访问，通过GitHub Action自动化渲染。

如果你在实际操作中遇到任何问题，欢迎在评论区留言。